In [ ]:
import subprocess
import os

netlist_template = """
*CONDUCTION NMOS W1

.INCLUDE ../../model/32nm_HP.pm
.OPTIONS GMIN=1e-020 ABSTOL=1e-18

*Definizione dei parametri
.PARAM Lmin=22n
.PARAM Wmin=22n
.PARAM Ldiff=44n

.TEMP 85

.PARAM NandP=4
.PARAM NandN=2
.PARAM NorP=4
.PARAM InvP=2

*Section A
*p3n + p2n 
A_Nor1_Mna A_Nor1_out P3n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
A_Nor1_Mnb A_Nor1_out P2n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
A_Nor1_Mpa A_Nor1_out P3n A_Nor1_Mpa_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
A_Nor1_Mpb A_Nor1_Mpa_x P2n vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*p1n + p0n 
A_Nor2_Mna A_Nor2_out P1n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
A_Nor2_Mnb A_Nor2_out P0n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
A_Nor2_Mpa A_Nor2_out P1n A_Nor2_Mpa_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
A_Nor2_Mpb A_Nor2_Mpa_x P0n vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*p3n + p2n + p1n + p0n = P
A_Nor3_Mna P A_Nor1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
A_Nor3_Mnb P A_Nor2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
A_Nor3_Mpa P A_Nor1_out A_Nor3_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
A_Nor3_Mpb A_Nor3_x A_Nor2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*Pn
A_Not1_Mp Pn P vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
A_Not1_Mn Pn P 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*Section B
*g2n + g3n 
B_Nand1_Mna B_Nand1_out G2n B_Nand1_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand1_Mnb B_Nand1_x G3n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand1_Mpa B_Nand1_out G2n vdd B_Nand1_out vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand1_Mpb B_Nand1_out G3n vdd B_Nand1_out vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g0n + g1n
B_Nand2_Mna B_Nand2_out G0n B_Nand2_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand2_Mnb B_Nand2_x G1n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand2_Mpa B_Nand2_out G0n vdd B_Nand2_out vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand2_Mpb B_Nand2_out G1n vdd B_Nand2_out vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + g1n + g0n + g3n
B_Nand3_Mna B_Nand3_out B_Nand1_out B_Nand3_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand3_Mnb B_Nand3_x B_Nand2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand3_Mpa B_Nand3_out B_Nand1_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand3_Mpb B_Nand3_out B_Nand2_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g2n + g1n + g0n + g3n)n
B_Not1_Mp B_Not1_out B_Nand3_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
B_Not1_Mn B_Not1_out B_Nand3_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*g2n + p1n 
B_Nand4_Mna B_Nand4_out G1n B_Nand4_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand4_Mnb B_Nand4_x P1n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand4_Mpa B_Nand4_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand4_Mpb B_Nand4_out P1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + g3n
B_Nand5_Mna B_Nand5_out G2n B_Nand5_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand5_Mnb B_Nand5_x G3n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand5_Mpa B_Nand5_out G2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand5_Mpb B_Nand5_out G3n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + g1n + g3n + p1n
B_Nand6_Mna B_Nand6_out B_Nand4_out B_Nand6_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand6_Mnb B_Nand6_x B_Nand5_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nand6_Mpa B_Nand6_out B_Nand4_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand6_Mpb B_Nand6_out B_Nand5_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g2n + g1n + g3n + p1n)n
B_Not2_Mp B_Not2_out B_Nand6_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
B_Not2_Mn B_Not2_out B_Nand6_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*g3n + g2n 
B_Nand7_Mna B_Nand7_out G3n B_Nand7_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
B_Nand7_Mnb B_Nand7_x G2n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
B_Nand7_Mpa B_Nand7_out G3n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand7_Mpb B_Nand7_out G2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + g3n + p2n = going to not1
B_Nand8_Mna B_Nand8_out B_Nand7_out B_Nand8_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
B_Nand8_Mnb B_Nand8_x P2n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
B_Nand8_Mpa B_Nand8_out B_Nand7_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand8_Mpb B_Nand8_out P2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g3n + p2n + g2n)n
B_Not3_Mp B_Not3_out B_Nand8_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
B_Not3_Mn B_Not3_out B_Nand8_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*p3n+g3n
B_Nand9_Mna B_Nand9_out P3n B_Nand9_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
B_Nand9_Mnb B_Nand9_x G3n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
B_Nand9_Mpa B_Nand9_out P3n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
B_Nand9_Mpb B_Nand8_out G3n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(p3n + g3n)n
B_Not4_Mp B_Not4_out B_Nand9_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
B_Not4_Mn B_Not4_out B_Nand9_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*(g2n + g1n + g0n + g3n)n NOR (g2n + g1n + g3n + p1n)n
B_Nor1_Mna B_Nor1_out B_Not1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nor1_Mnb B_Nor1_out B_Not2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nor1_Mpa B_Nor1_out B_Not1_out B_Nor1_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
B_Nor1_Mpb B_Nor1_x B_Not2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*(g3n + p2n + g2n)n NOR (p3n + g3n)n
B_Nor2_Mna B_Nor2_out B_Not3_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nor2_Mnb B_Nor2_out B_Not4_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nor2_Mpa B_Nor2_out B_Not3_out B_Nor2_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
B_Nor2_Mpb B_Nor2_x B_Not4_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*(g2n + g1n + g0n + g3n)n NOR (g2n + g1n + g3n + p1n)n *NOR* (g3n + p2n + g2n)n NOR (p3n + g3n)n = G
B_Nor3_Mna G B_Nor1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nor3_Mnb G B_Nor2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
B_Nor3_Mpa G B_Nor1_out B_Nor3_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
B_Nor3_Mpb B_Nor3_x B_Nor2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*Gn
B_Not5_Mp Gn G vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
B_Not5_Mn Gn G 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}


*Section C
*g2n + g1n 
C_Nand1_Mna C_Nand1_out G2n C_Nand1_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand1_Mnb C_Nand1_x G1n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand1_Mpa C_Nand1_out G2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand1_Mpb C_Nand1_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g0n + cnn
C_Nand2_Mna C_Nand2_out G0n C_Nand2_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand2_Mnb C_Nand2_x Cnn 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand2_Mpa C_Nand2_out G0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand2_Mpb C_Nand2_out Cnn vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + g1n + g0n + cnn
C_Nand3_Mna C_Nand3_out C_Nand1_out C_Nand3_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand3_Mnb C_Nand3_x C_Nand2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand3_Mpa C_Nand3_out C_Nand1_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand3_Mpb C_Nand3_out C_Nand2_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g2n + g1n + g0n + cnn)n
C_Not1_Mp C_Not1_out C_Nand3_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
C_Not1_Mn C_Not1_out C_Nand3_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*g2n + g1n 
C_Nand4_Mna C_Nand4_out G2n C_Nand4_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand4_Mnb C_Nand4_x G1n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand4_Mpa C_Nand4_out G2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand4_Mpb C_Nand4_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g0n + p0n
C_Nand5_Mna C_Nand5_out G0n C_Nand5_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand5_Mnb C_Nand5_x P0n 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand5_Mpa C_Nand5_out G0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand5_Mpb C_Nand5_out P0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + g1n + g0n + cnn
C_Nand6_Mna C_Nand6_out C_Nand4_out C_Nand6_x 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand6_Mnb C_Nand6_x C_Nand5_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nand6_Mpa C_Nand6_out C_Nand4_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand6_Mpb C_Nand6_out C_Nand5_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g2n + g1n + g0n + p0n)n
C_Not2_Mp C_Not2_out C_Nand6_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
C_Not2_Mn C_Not2_out C_Nand6_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*g1n + p1n 
C_Nand7_Mna C_Nand7_out P1n C_Nand7_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
C_Nand7_Mnb C_Nand7_x G1n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
C_Nand7_Mpa C_Nand7_out P1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand7_Mpb C_Nand7_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*g2n + p1n + g1n = going to not1
C_Nand8_Mna C_Nand8_out C_Nand7_out C_Nand8_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
C_Nand8_Mnb C_Nand8_x G2n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
C_Nand8_Mpa C_Nand8_out C_Nand7_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand8_Mpb C_Nand8_out G2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g1n + p1n + g2n)n
C_Not3_Mp C_Not3_out C_Nand8_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
C_Not3_Mn C_Not3_out C_Nand8_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*p2n+g2n
C_Nand9_Mna C_Nand9_out P2n C_Nand9_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
C_Nand9_Mnb C_Nand9_x G2n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
C_Nand9_Mpa C_Nand9_out P2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
C_Nand9_Mpb C_Nand8_out G2n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(p2n + g2n)n
C_Not4_Mp C_Not4_out C_Nand9_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
C_Not4_Mn C_Not4_out C_Nand9_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*(g2n + g1n + g0n + cnn)n NOR (g2n + g1n + g0n + p0n)n
C_Nor1_Mna C_Nor1_out C_Not1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nor1_Mnb C_Nor1_out C_Not2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nor1_Mpa C_Nor1_out C_Not1_out C_Nor1_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
C_Nor1_Mpb C_Nor1_x C_Not2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*(g2n + g1n + p1n)n NOR (g2n + p2n)n
C_Nor2_Mna C_Nor2_out C_Not3_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nor2_Mnb C_Nor2_out C_Not4_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nor2_Mpa C_Nor2_out C_Not3_out C_Nor2_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
C_Nor2_Mpb C_Nor2_x C_Not4_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*(g2n + g1n + p1n)n NOR (g2n + p2n)n *NOR* (g2n + g1n + g0n + cnn)n NOR (g2n + g1n + g0n + p0n)n = cnz
C_Nor3_Mna Cnz C_Nor1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nor3_Mnb Cnz C_Nor2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
C_Nor3_Mpa Cnz C_Nor1_out C_Nor3_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
C_Nor3_Mpb C_Nor3_x C_Nor2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*Section D
*cnn + g0n 
D_Nand1_Mna D_Nand1_out Cnn D_Nand1_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand1_Mnb D_Nand1_x G0n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand1_Mpa D_Nand1_out Cnn vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
D_Nand1_Mpb D_Nand1_out G0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*cnn + g0n + g1n = going to not1
D_Nand2_Mna D_Nand2_out D_Nand1_out D_Nand2_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand2_Mnb D_Nand2_x G1n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand2_Mpa D_Nand2_out D_Nand1_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
D_Nand2_Mpb D_Nand2_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(cnn + g0n + g1n)n
D_Not1_Mp D_Not1_out D_Nand2_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
D_Not1_Mn D_Not1_out D_Nand2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*p0n + g1n 
D_Nand3_Mna D_Nand3_out P0n D_Nand3_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand3_Mnb D_Nand3_x G1n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand3_Mpa D_Nand3_out P0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
D_Nand3_Mpb D_Nand3_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*p0n + g0n + g1n = going to not2
D_Nand4_Mna D_Nand4_out D_Nand3_out D_Nand4_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand4_Mnb D_Nand4_x G0n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand4_Mpa D_Nand4_out D_Nand3_out vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
D_Nand4_Mpb D_Nand4_out G0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(p0n + g0n + g1n)n
D_Not2_Mp D_Not2_out D_Nand4_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
D_Not2_Mn D_Not2_out D_Nand4_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*(p0n + g0n + g1n)n NOR (cnn + g0n + g1n)n
D_Nor1_Mna D_Nor1_out D_Not1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
D_Nor1_Mnb D_Nor1_out D_Not2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
D_Nor1_Mpa D_Nor1_out D_Not1_out D_Nor1_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
D_Nor1_Mpb D_Nor1_x D_Not2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*p1n + g1n 
D_Nand5_Mna D_Nand5_out P1n D_Nand5_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand5_Mnb D_Nand5_x G1n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
D_Nand5_Mpa D_Nand5_out P1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
D_Nand5_Mpb D_Nand5_out G1n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(p1n+g1n)n
D_Not3_Mp D_Not3_out D_Nand5_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
D_Not3_Mn D_Not3_out D_Nand5_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*((p0n + g0n + g1n)n NOR (cnn + g0n + g1n)n) NOR (p1n+g1n)n = Cny
D_Nor2_Mna Cny D_Nor1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
D_Nor2_Mnb Cny D_Not3_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
D_Nor2_Mpa Cny D_Nor1_out D_Nor2_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
D_Nor2_Mpb D_Nor2_x D_Not3_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}

*Section E 
*g0n + Cnn 
E_Nand1_Mna E_Nand1_out G0n E_Nand1_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
E_Nand1_Mnb E_Nand1_x Cnn 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
E_Nand1_Mpa E_Nand1_out G0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
E_Nand1_Mpb E_Nand1_out Cnn vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g0n +Cnn)n
E_Not1_Mp E_Not1_out E_Nand1_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
E_Not1_Mn E_Not1_out E_Nand1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*g0n + p0n 
E_Nand2_Mna E_Nand2_out G0n E_Nand2_x 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
E_Nand2_Mnb E_Nand2_x P0n 0 0 nmos W={Wmin*NandN} L={Lmin} AS={Wmin*NandN} AD={Wmin*NandN*Ldiff} PS={2*(Ldiff+Wmin*NandN)} PD={2*(Ldiff+Wmin*NandN)}
E_Nand2_Mpa E_Nand2_out G0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}
E_Nand2_Mpb E_Nand2_out P0n vdd vdd pmos W={Wmin*NandP} L={Lmin} AS={Wmin*NandP*Ldiff} AD={Wmin*NandP*Ldiff} PS={2*(Ldiff+Wmin*NandP)} PD={2*(Ldiff+Wmin*NandP)}

*(g0n +p0n)n
E_Not2_Mp E_Not2_out E_Nand2_out vdd vdd pmos W={Wmin*InvP} L={Lmin} AS={Wmin*InvP*Ldiff} AD={Wmin*InvP*Ldiff} PS={2*(Ldiff+Wmin*InvP)} PD={2*(Ldiff+Wmin*InvP)}
E_Not2_Mn E_Not2_out E_Nand2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}

*(g0n +p0n)n NOR (g0n +Cnn)n = cn+x
E_Nor1_Mna Cnx E_Not1_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
E_Nor1_Mnb Cnx E_Not2_out 0 0 nmos W={Wmin} L={Lmin} AS={Wmin*Ldiff} AD={Wmin*Ldiff} PS={2*(Ldiff+Wmin)} PD={2*(Ldiff+Wmin)}
E_Nor1_Mpa Cnx E_Not1_out E_Nor1_x vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}
E_Nor1_Mpb E_Nor1_x E_Not2_out vdd vdd pmos W={Wmin*NorP} L={Lmin} AS={Wmin*NorP*Ldiff} AD={Wmin*NorP*Ldiff} PS={2*(Ldiff+Wmin*NorP)} PD={2*(Ldiff+Wmin*NorP)}


VG0n G0n 0 


*Definizione del tipo di analisi
.CONTROL
dc TEMP 85 86 10

.ENDC
.END

"""

inp1s = [0, 'alim', 'alim', 0]
inp2s = [0, 'alim', 0, 'alim']

drains = ['alim', 0, 'alim', 'alim']

output_dir = "diff_inputs"
total_numbers = 2 ** 9

for number in range(total_numbers):
    binary_representation = format(number, '09b')
    Cnn, P0n, P1n, P2n, P3n, G0n, G1n, G2n, G3n = [int(digit) for digit in binary_representation]
    


# make the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

outs = []
i = 0
for inp1, inp2 in zip(inp1s, inp2s):
    netlist_content = netlist_template.format(drain_v=drains[i], inp1=inp1, inp2=inp2)
    i += 1
    
    filename = f"{output_dir}/nmos_{inp1},{inp2}.net"
                                                                                                                                                                                                    
    with open(filename, 'w') as file:
        file.write(netlist_content)
    # add a -b and -o flag also pipe it to a logs.txt file
    p = subprocess.run(["ngspice", filename] , encoding='utf-8', stdout=subprocess.PIPE)
    windstr = p.stdout
    outs.append(windstr)

    
      

In [48]:
matrix = []
for o in outs:
    lines = o.split('\n')
    matrix.append(lines[10])
    
print(matrix)

['v(drain_inter) = 8.413042e-18', 'v(drain_inter) = -1.20882e-18', 'v(drain_inter) = 5.750114e-18', 'v(drain_inter) = 1.738211e-15']


In [49]:
inp1s = [0, 1, 1, 0]
inp2s = [0, 1, 0, 1]

filename = f"outputs.csv"
with open(filename, 'w') as file:
    file.write("inp1,inp2,V(Internal node)\n")
    for i, line in enumerate(matrix):
        file.write(f"{inp1s[i]},{inp2s[i]},{line.split(' ')[-1]}\n")